In [1]:
!pip install pyspark faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=76a6a12eef6d2757244fa030e654a9005cc8b36222838f72902ed1c0f235e2eb
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [51]:
from pyspark.sql import SparkSession
from faker import Faker
import random
from datetime import datetime, timedelta

# Criando sessão Spark e instanciando o Faker
spark = SparkSession.builder.appName('bank_balances').getOrCreate()
fake = Faker()

def generate_data(n):
    data = []
    for _ in range(n):
        name = fake.name()
        balance = round(random.uniform(1000, 10000), 2)
        account_number = fake.bban()
        city = fake.city()
        state = fake.state()
        birth_date = fake.date_of_birth(minimum_age=18, maximum_age=80)  # Mantém como datetime
        age = datetime.today().year - birth_date.year  # Calcula a idade

        # Calcula a data a partir dos 18 anos do cliente
        min_account_create_date = birth_date + timedelta(days=18*365)

        # Gera a data de criação da conta, começando a partir dos 18 anos do cliente
        create_account_date = fake.date_between(start_date=min_account_create_date, end_date='today')
        account_age = datetime.today().year - create_account_date.year  # Calcula a idade da conta

        # Agora formatamos create_account_date para string depois de calcular a idade da conta
        create_account_date_str = create_account_date.strftime('%d-%m-%Y')

        # Montamos os dados e adicionamos à lista
        data.append((name, balance, account_number, city, age, account_age, create_account_date_str, birth_date.strftime('%d-%m-%Y')))  # Converte a data de nascimento para string
    return data

# Gerando os dados
dados = generate_data(50000)

# Definindo o schema para o DataFrame
colunas = ['name', 'balance', 'account_number', 'city', 'age', 'account_age', 'create_account_date', 'birth_date']

# Criando o DataFrame PySpark
df = spark.createDataFrame(dados, schema=colunas)

# Exibindo os 10 primeiros registros
df.show(10)


+-------------------+-------+------------------+----------------+---+-----------+-------------------+----------+
|               name|balance|    account_number|            city|age|account_age|create_account_date|birth_date|
+-------------------+-------+------------------+----------------+---+-----------+-------------------+----------+
|       Rachel Jones|2020.51|MTQF70972047414106|     Michaelberg| 66|         29|         18-01-1995|17-02-1958|
|      Jacob Flowers|3204.92|GEAT95699522480569|      Harmonside| 81|         25|         04-07-1999|27-10-1943|
|      Joseph Morgan|8674.33|SGJD66529557897484|    Michellebury| 19|          1|         09-11-2023|11-08-2005|
|     Crystal Steele|5129.72|FNRB00054006263651| Port Josephtown| 69|         41|         16-08-1983|08-05-1955|
|      Kevin Johnson| 8759.1|VVJO57967779343736|     Carrieville| 57|          7|         15-07-2017|28-08-1967|
|Regina Contreras MD|5573.53|PSAW84626641264294|West Chelseafurt| 44|         22|         12-06-

In [52]:
df.count()

50000

In [53]:
# ordenação dos dados pelo saldo

df_ordenado = df.orderBy(df.balance.desc()).limit(10)

# <.limit(10)> limita a 10 o tamanho da lista... podendo ser usado qualquer valor.

df_ordenado.show()

+------------------+-------+------------------+----------------+---+-----------+-------------------+----------+
|              name|balance|    account_number|            city|age|account_age|create_account_date|birth_date|
+------------------+-------+------------------+----------------+---+-----------+-------------------+----------+
|Debbie Herrera DDS|9999.86|CVWQ55936715002212|       East Todd| 32|          8|         22-05-2016|30-12-1992|
|     Caitlin Stark|9999.71|BZJZ52392250408146|New Kristineport| 28|          1|         18-05-2023|11-04-1996|
|Kathleen Contreras| 9999.5|KCYK58564922964783|       Quinnland| 53|          9|         08-02-2015|27-06-1971|
|      Rachel Davis|9999.21|DGWS36189950316315|  Hernandezmouth| 26|          5|         10-08-2019|27-08-1998|
|    Paige Chandler|9998.93|FVXZ54735928563961|  East Terryland| 60|          6|         21-12-2018|11-10-1964|
|   Sheri Hernandez|9998.89|SABX30282279069427|      Ritterbury| 39|         16|         11-03-2008|08-0

In [54]:
# Busca de contas com saldo maior que um valor específico.

valor = 5000

df_valor_acima = df.filter(df.balance > valor)

df_valor_acima.show()

+-------------------+-------+------------------+-----------------+---+-----------+-------------------+----------+
|               name|balance|    account_number|             city|age|account_age|create_account_date|birth_date|
+-------------------+-------+------------------+-----------------+---+-----------+-------------------+----------+
|      Joseph Morgan|8674.33|SGJD66529557897484|     Michellebury| 19|          1|         09-11-2023|11-08-2005|
|     Crystal Steele|5129.72|FNRB00054006263651|  Port Josephtown| 69|         41|         16-08-1983|08-05-1955|
|      Kevin Johnson| 8759.1|VVJO57967779343736|      Carrieville| 57|          7|         15-07-2017|28-08-1967|
|Regina Contreras MD|5573.53|PSAW84626641264294| West Chelseafurt| 44|         22|         12-06-2002|03-05-1980|
|    Anthony Gregory|8711.58|IBUW16014678311865|     Crawfordberg| 46|         23|         17-01-2001|24-07-1978|
|      Michael Green|9758.25|HKWA85091535742643|     Mitchellport| 66|          1|      

In [55]:
df_valor_acima.count()

27749

In [62]:
# Busca de cliente com uma idade maior que uma específica.

idade = 60

df_idade_acima = df.filter(df.age > idade)

df_idade_acima.show(10)
print('-----'*50)
print('clientes acima de', idade, "=", df_idade_acima.count(), 'clientes')

+--------------+-------+------------------+----------------+---+-----------+-------------------+----------+
|          name|balance|    account_number|            city|age|account_age|create_account_date|birth_date|
+--------------+-------+------------------+----------------+---+-----------+-------------------+----------+
|  Rachel Jones|2020.51|MTQF70972047414106|     Michaelberg| 66|         29|         18-01-1995|17-02-1958|
| Jacob Flowers|3204.92|GEAT95699522480569|      Harmonside| 81|         25|         04-07-1999|27-10-1943|
|Crystal Steele|5129.72|FNRB00054006263651| Port Josephtown| 69|         41|         16-08-1983|08-05-1955|
| Michael Green|9758.25|HKWA85091535742643|    Mitchellport| 66|          1|         16-04-2023|21-12-1958|
| Eddie Bennett|1724.83|CNFA97033332199838|      New Marcus| 66|         19|         24-10-2005|26-03-1958|
|Brittany Davis|7135.53|SKUN62666000033001|     Port Jeremy| 65|         39|         28-12-1985|11-08-1959|
|  Janet Burton|6361.82|SLPF

In [61]:
# Definindo os limites de idade e criando a condição para saldo maior
idade_min = 30
idade_max = 50
idade_min_conta = 10

# Filtrando os clientes com idade entre 30 e 50 anos e idade da conta menor ou igual a 10 anos
df_clientes_filtrados = df.filter(
    (df.age >= idade_min) &
    (df.age <= idade_max) &
    (df.account_age >= idade_min_conta)
).orderBy(df.balance.desc())

# Exibindo os 10 clientes com maior saldo dentro da faixa etária
df_clientes_filtrados.show(10)

# Exibindo o número total de clientes nessa condição
print('-----' * 50)
print()
print('Total de clientes entre', idade_min, 'e', idade_max, 'anos', 'com conta a mais de', idade_min_conta, 'anos =', df_clientes_filtrados.count(), 'clientes')


+-----------------+-------+------------------+------------------+---+-----------+-------------------+----------+
|             name|balance|    account_number|              city|age|account_age|create_account_date|birth_date|
+-----------------+-------+------------------+------------------+---+-----------+-------------------+----------+
|  Sheri Hernandez|9998.89|SABX30282279069427|        Ritterbury| 39|         16|         11-03-2008|08-02-1985|
|Mr. Ronald Rivera|9998.83|ETHZ16727588350212|   South Cassandra| 44|         23|         18-12-2001|27-11-1980|
|      Tina Foster|9997.47|EXXV62472888209282|    East Christina| 41|         10|         01-07-2014|08-09-1983|
|      Ronald Ward|9996.97|VXRX18804312720427|       West Daniel| 33|         10|         03-12-2014|16-02-1991|
|       Mario Bell| 9996.8|BTGQ01585058489421|     New Alexandra| 35|         11|         14-07-2013|18-06-1989|
| Amanda Carpenter|9996.57|SFWG68648572661240|West Danielleshire| 35|         12|         25-05-